In [1]:
!pip install clingo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.3 MB/s eta 0:00:0000:0100:01


In [2]:
### DO NOT CHANGE THIS CELL

%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

import clingo
from IPython.display import clear_output

ModuleNotFoundError: No module named 'networkx'

In [ ]:
### YOU MAY CHANGE THIS CELL TO TEST YOUR SOLUTION ON DIFFERENT INPUTS

nodes = list(range(1, 6+1))
edges = [
    (1,2),
    (2,3),
    (3,4),
    (4,5),
    (2,6),
    (3,6),
]
factories = [
    {
        'location': 1,
        'load': 10,
    },
]
delivery_points = [
    {
        'location': 4,
        'demand': 2,
    },
    {
        'location': 5,
        'demand': 2,
    },
]
charging_points = [
    {
        'location': 2,
    },
    {
        'location': 6,
    },
]
trucks = [
    {
        'location': 1,
        'load': 0,
        'capacity': 1,
        'charge level': 2,
        'charge capacity': 10,
    },
    {
        'location': 2,
        'load': 0,
        'capacity': 1,
        'charge level': 2,
        'charge capacity': 10,
    },
]

num_time_steps = 40

In [ ]:
### DO NOT CHANGE THIS CELL

warehouse_locations = [
    warehouse_info['location']
    for warehouse_info in factories
]
truck_locations = [
    truck_info['location']
    for truck_info in trucks
]
delivery_locations = [
    dp_info['location']
    for dp_info in delivery_points
]
charging_locations = [
    cp_info['location']
    for cp_info in charging_points
]

class InvalidInputError(Exception):
    pass

intersection = [location for location in warehouse_locations if location in delivery_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a warehouse and a delivery location')

intersection = [location for location in warehouse_locations if location in charging_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a warehouse and a charging location')

intersection = [location for location in delivery_locations if location in charging_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a delivery and a charging location')

In [ ]:
# Load example inputs 
asp_program = ""
asp_program += f"     #const t={num_time_steps}.\n"
asp_program += f"     #const tr={len(trucks)}.\n"
asp_program += f"     #const charge_cap={trucks[0]['charge capacity']}.\n" #all trucks have the same battery capacity safe to make that assumption??
for node in nodes:
  asp_program += f"     node({node}).\n"

for edge in edges:
  asp_program += f"     edge({edge[0]},{edge[1]}).\n"

max_factory_load = 0
for i, factory in enumerate(factories):
  location = factory['location']
  load = factory['load']
  if load >max_factory_load:
    max_factory_load = load
  asp_program += f"     factory_loc({location}).\n"
  asp_program += f"     factory_load({location},{load}).\n"

asp_program += f"     #const loads_f={max_factory_load}.\n"

max_delivery_load = 0
for d_point in delivery_points:
  location = d_point['location']
  demand = d_point['demand']
  if demand >max_delivery_load:
    max_delivery_load = demand
  asp_program += f"     delivery_point({location}).\n"
  asp_program += f"     delivery_demand({location},{demand}).\n"

asp_program += f"     #const loads_d={max_delivery_load }.\n"

for c_point in charging_points:
  location = c_point['location']
  asp_program += f"     charging_point({location}).\n"

max_load_for_truck = 0
for i, truck in enumerate(trucks):
  location = truck['location']
  load = truck['load']
  capacity = truck['capacity']
  if capacity > max_load_for_truck:
    max_load_for_truck = capacity
  c_level = truck['charge level']
  c_capacity = truck['charge capacity']
  asp_program += f"     truck({i+1}).\n"
  asp_program += f"     charge_level({c_level}).\n"
  asp_program += f"     truck_at({i+1},{location}).\n"
  asp_program += f"     truck_load({i+1},{load}).\n"
  asp_program += f"     truck_capacity({i+1},{capacity}).\n"
  asp_program += f"     truck_charge_level({i+1},{c_level}).\n"
  asp_program += f"     charge_capacity({i+1},{c_capacity}).\n" 

asp_program += f"     #const loads_t={max_load_for_truck}.\n"

In [ ]:
asp_program += """

    % Generate times till maximum number of time steps
    time(1..t).
    %possible_time(1..t).
    %{ time(T) : possible_time(T) }.
    %:- not time(1). %first time step is time 1
    %:- not time(T), time(T+1), possible_time(T), possible_time(T+1).

    charge_levels(1..charge_cap).
    load_for_trucks(0..loads_t).
    load_for_f(0..loads_f).
    load_for_d(0..loads_d).

    %*{ time(T) : possible_time(T) }.
    :- not time(1). %first time step is time 1
    :- possible_time(T), possible_time(T+1), not time(T), time(T+1). %times are sequential
    *%

    % Max time is last time step
    %max_time(MaxTime) :- MaxTime = #max { T : time(T) }.

    % Edges are undirected
    edge(X,Y) :- edge(Y,X).

    % These are the possible actions that we can take
    action(wait).
    action(move(X,Y)) :- edge(X,Y).
    action(load).
    action(unload).
    action(charge).

    % We perform exactly one action at each time step
    1 { do(T, Tr, A) : action(A) } 1 :- time(T), T <t,  truck(Tr).
    
    """

In [ ]:
asp_program += """
    truck_at(Tr, Loc) :-  do(T, Tr, wait), truck_at(Tr, Loc), truck(Tr), node(Loc).

    truck_at(Tr, Y) :- do(T, Tr, move(X,Y)), truck_at(Tr, X), truck(Tr), node(Y).

    %truck_charge_level(Tr, L-1) :-  do(T, Tr, move(_,_)), truck_charge_level(Tr, L), truck(Tr), charge_levels(L-1), charge_levels(L).
    %truck_charge_level(Tr, L+1) :-  do(T, Tr, charge), truck(Tr), truck_charge_level(Tr, L), charge_levels(L+1), charge_levels(L), truck_at(Tr, Loc).
    truck_at(Tr, Loc) :- do(T, Tr, charge), truck(Tr), truck_at(Tr, Loc), charging_point(Loc), truck_charge_level(Tr, L+1).

    truck_load(Tr, L-1) :-  do(T, Tr, unload), truck_load(Tr, L), truck(Tr), load_for_trucks(L-1), load_for_trucks(L).
    delivery_demand(Loc, L-1) :-  do(T, _, unload), delivery_demand(Loc, L),  truck_at(_, Loc), load_for_d(L-1), load_for_d(L).

  
    factory_load(F, L-1) :-  do(T, Tr, load), factory_load(F, L), truck_at(Tr, F), factory_loc(F), load_for_f(L-1), load_for_f(L).
    truck_load(Tr, L+1) :-  do(T, Tr, load), truck_load(Tr, L), truck(Tr), load_for_trucks(L-1), load_for_trucks(L).
  
  
"""

In [ ]:
asp_program += """
    % Cannot move when battery is empty
    :- do(T, Tr, move(_,_)), truck_charge_level(Tr, 0), time(T).

    % At each location, there can be at most one truck at the same time.
    :- truck_at(Tr1, Loc), truck_at(Tr2, Loc), Tr1!=Tr2.

    % Trucks cannot pass each other on a piece of road—e.g., trucks at adjacent locations cannot swap places in a single time step.
    :- do(T, Tr1, move(X,Y)), do(T, Tr2, move(Y, X)), truck_at(Tr1, X), truck_at(Tr2,Y), Tr1 != Tr2, time(T).

    % Cannot move to node without an edge
    :- do(T, Tr, move(X,Y)), truck_at(Tr, X), not edge(X,Y), time(T).

    % Cannot charge if truck is not in charging location
    :- do(T, Tr, charge),  truck_at(Tr, Loc), not charging_point(Loc), time(T).

    % Cannot charge when battery is full
    :- do(T,Tr, charge), truck_charge_level(Tr, charge_cap), time(T).

     % Cannot unload if not at delivery point
    :- do(T, Tr, unload), truck_at(Tr, Loc), not delivery_point(Loc), time(T).

    % Cannot unload if the demand is 0
    :- do(T, Tr, unload), truck_at(Tr, Loc), delivery_demand(Loc, 0), time(T).

    % Cannot unload when truck load is 0
    :- do(T, Tr, unload),  truck_load(Tr, 0), time(T).

    % Cannot load if not at factory
    :- do(T, Tr, load), truck_at(Tr, Loc), not factory_loc(Loc), time(T).

    % Cannot load if the factory is empty
    :- do(T, Tr, load), truck_at(Tr, Loc),  factory_load(Loc, 0), time(T).

    % Cannot load when load capacity is full
    :- do(T, Tr, load), truck_load(Tr,load), truck_capacity(Tr, cap), load>=cap, time(T).

    % A truck cannot be left without enough battery to reach a charging point
    :- truck_at(Tr, X), truck_charge_level(Tr, L), charging_point_dist(X, DP), L<DP.

    :- do(T, Tr, wait), do(T+1, Tr, wait), time(T), time(T+1), T+1 < t.

"""

In [ ]:
asp_program += """

    all_distances(0..M) :- M = #max{N : node(N)}.
    distance(X,X,0) :- node(X).
    distance(X, Y, D+1) :- all_distances(D+1), distance(X, Z, D), edge(Z,Y), not distance(X,Y, DMin) : all_distances(DMin), DMin<=D.

    charging_point_dist(X, PD) :- node(X), PD = #min{ D, Y: distance(X,Y,D), charging_point(Y)}.

    deliveries_done(T, Loc) :- time(T), delivery_demand(Loc,0).
    truck_parked(T, Tr) :- time(T), truck_at(Tr, Loc), charging_point(Loc).
    truck_empty(T, Tr) :- time(T), truck_load(Tr,0).
    find_ch_point(T, Tr) :- time(T), truck_at(Tr, Loc), truck_charge_level(Tr, L), shortest_path(Loc, Node), Node<=L.

    goal_reached(T) :- time(T),
	      deliveries_done(T, LocD): delivery_point(LocD);
        truck_parked(T, Tr): truck(Tr);
        truck_empty(T, Tr): truck(Tr) ;
        find_ch_point(T, Tr): truck(Tr).
    
    %:- not goal_reached(t). %, time(t).
    %:- time(M), goal_reached(t), M>t.

    %find a condition that doesnt allow all trucks to wait at one timestep
    %:- do(T, Tr1, wait), do(T, Tr2, wait), Tr1!=Tr2.

    %#minimize { 1,T : time(T) }.
    %#minimize { 1,T : do(T, _, wait) }.
    #show do/3.
       

"""

In [ ]:
print(asp_program)

     #const t=40.
     #const tr=2.
     #const charge_cap=10.
     node(1).
     node(2).
     node(3).
     node(4).
     node(5).
     node(6).
     edge(1,2).
     edge(2,3).
     edge(3,4).
     edge(4,5).
     edge(2,6).
     edge(3,6).
     factory_loc(1).
     factory_load(1,10).
     #const loads_f=10.
     delivery_point(4).
     delivery_demand(4,2).
     delivery_point(5).
     delivery_demand(5,2).
     #const loads_d=2.
     charging_point(2).
     charging_point(6).
     truck(1).
     charge_level(2).
     truck_at(1,1).
     truck_load(1,0).
     truck_capacity(1,1).
     truck_charge_level(1,2).
     charge_capacity(1,10).
     truck(2).
     charge_level(2).
     truck_at(2,2).
     truck_load(2,0).
     truck_capacity(2,1).
     truck_charge_level(2,2).
     charge_capacity(2,10).
     #const loads_t=1.


    % Generate times till maximum number of time steps
    time(1..t).
    %possible_time(1..t).
    %{ time(T) : possible_time(T) }.
    %:- not time(1). %first t

In [ ]:
### DO NOT CHANGE THIS CELL

# Some machinery to (dynamically) print things in the notebook
log_str = ""
def reset_log():
    global log_str
    log_str = ""
def log(msg):
    global log_str
    print(msg, flush=True)
    log_str += f"{msg}\n"
def print_log():
    global log_str
    print(log_str, end='', flush=True)
    
def display_solution(model):
    """
    Displays a solution specified by the answer set
    in a human-readable format.
    
    Assumes that the answer set program uses do/3
    to specify actions, and uses truck_at/2 to
    specify the initial locations of the trucks.
    """
    actions = {}
    truck_locations = {}
    for atom in model.symbols(atoms=True):
        if atom.name == "do":
            time = atom.arguments[0].number
            truck = str(atom.arguments[1])
            action = str(atom.arguments[2])
            print(time, truck, action)
            if time not in actions:
                actions[time] = {}
            actions[time][truck] = action
        elif atom.name == "truck_at":
            truck_locations[str(atom.arguments[0])] = str(atom.arguments[1])
    solution_str = "\n"
    solution_str += " "*5 + "".join([
        f"{truck:<5} @ {truck_locations[truck]:<10}"
        for truck in sorted(truck_locations)
    ]) + "\n"
    solution_str += "-"*(5+18*len(truck_locations)) + "\n"
    for time in sorted(actions):
        time_str = f"{time}:"
        solution_str += f"{time_str:<4} " + "".join([
            f"{actions[time][truck]:<18}"
            for truck in sorted(truck_locations)
        ]) + "\n"
    return solution_str

def find_and_output_solution(
    asp_program,
    timeout=10,
    num_threads=1,
):
    """
    Calls the ASP solver to find answer sets,
    and translates these to solutions,
    which are printed in human-readable format.
    """

    reset_log()
    log("..Grounding..")
    
    control = clingo.Control([f"--parallel-mode={num_threads}"])
    control.add("base", [], asp_program)
    control.ground([("base", [])])

    control.configuration.solve.models = 1
    control.configuration.solve.opt_mode = "optN"
    
    def interpret_model(model):
        clear_output(wait=True)
        print_log()
        sol = display_solution(model)
        print(sol)

    if timeout > 0:
        log(f"..Solving with timeout {timeout}s..")
        with control.solve(
            async_=True,
            on_model=lambda model: interpret_model(model),
        ) as handle:
            finished = handle.wait(timeout)
            if not finished:
                log("..Stopped after timeout..")
            else:
                log("..Stopped..")
                time = control.statistics['summary']['times']['total']
                log(f"..Total solving time: {time:.2f} seconds..")
    else:
        log("..Solving..")
        control.solve(
            on_model=lambda model: interpret_model(model),
        )
        log("..Stopped..")
        time = control.statistics['summary']['times']['total']
        log(f"..Total solving time: {time:.2f} seconds..")

In [ ]:
find_and_output_solution(
    asp_program,
    timeout=0,
    num_threads=4,
)

..Grounding..
..Solving..
1 1 wait
2 1 wait
3 1 wait
4 1 wait
5 1 wait
6 1 wait
7 1 wait
8 1 wait
9 1 wait
10 1 wait
11 1 wait
12 1 wait
13 1 wait
14 1 wait
15 1 wait
16 1 wait
17 1 wait
18 1 wait
19 1 wait
20 1 wait
21 1 wait
22 1 wait
23 1 wait
24 1 wait
25 1 wait
26 1 wait
27 1 wait
28 1 wait
29 1 wait
30 1 wait
31 1 wait
32 1 wait
33 1 wait
34 1 wait
35 1 wait
36 1 wait
37 1 wait
38 1 wait
39 1 wait
1 2 wait
2 2 wait
3 2 wait
4 2 wait
5 2 wait
6 2 wait
7 2 wait
8 2 wait
9 2 wait
10 2 wait
11 2 wait
12 2 wait
13 2 wait
14 2 wait
15 2 wait
16 2 wait
17 2 wait
18 2 wait
19 2 wait
20 2 wait
21 2 wait
22 2 wait
23 2 wait
24 2 wait
25 2 wait
26 2 wait
27 2 wait
28 2 wait
29 2 wait
30 2 wait
31 2 wait
32 2 wait
33 2 wait
34 2 wait
35 2 wait
36 2 wait
37 2 wait
38 2 wait
39 2 wait

     1     @ 1         2     @ 2         
-----------------------------------------
1:   wait              wait              
2:   wait              wait              
3:   wait              wait              
4